<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-19 14:41:32
-------------------
qualified stocks: 94
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  1.01 L
Current:  1.37 C
-------------------
Today PnL: 1.15 L (0.84%)
Current PnL: -17.45 L (-11.87%)
CY Booked + Current PnL: -9.18 L (-6.25%)
-------------------
Total profit:  3.79 L
Total loss:  -21.24 L
-------------------
Total Booked + Current PnL: 17.80 L (14.82%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.02%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 77.07 L (56.09%)
Deployed:  1.20 C
Current:  1.37 C
CAGR/XIRR %: 8.66%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,756.01,-18.25,H-LC,6.80,113296.0,8984.0,7670.0,-0.85,8.61,6.77,15.96,9.0,1.17,0.83,26.97,X5K,ATH,METALS
69,SBIN,760.30,863.00,-11.34,M-LC,3.76,219831.0,18352.0,8859.0,0.23,9.11,4.03,13.51,60.0,2.07,1.61,22.85,XY25,NTT,BANKS
52,LALPATHLAB,2838.53,3545.00,-22.49,H-SC,11.63,231683.0,38663.0,9383.0,-0.63,20.03,4.05,24.89,137.0,4.12,1.70,48.05,X40N,NTT,HEALTHCARE
4,APOLLOHOSP,7098.95,8285.00,-13.64,H-LC,6.60,172458.0,16281.0,9813.0,0.34,10.42,5.69,16.71,22.0,1.66,1.26,29.68,X40N,BTT,HEALTHCARE
56,MEDANTA,1087.93,1486.00,11.18,H-MC,13.93,158700.0,32500.0,13680.0,1.85,25.75,8.62,36.59,108.0,2.38,1.16,41.49,XY24,NTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,RAJOOENG,102.50,143.33,-32.22,H-SC,7.60,102730.0,230.0,40599.0,1.08,0.22,39.52,39.83,114.0,0.01,0.75,1.71,AR,ATH,MISC
33,HAVELLS,1588.02,2069.16,-1.08,H-MC,3.53,245326.0,-2405.0,77474.0,1.29,-0.97,31.58,30.30,92.0,-0.03,1.80,12.19,X40,ATH,ELECTRICAL
12,BANKINDIA,116.91,190.00,-30.43,H-MC,7.53,178792.0,-1016.0,113426.0,1.21,-0.56,63.44,62.52,88.0,-0.01,1.31,33.33,XR,NTT,BANKS
59,PGHH,13388.00,18062.58,-31.21,H-MC,6.11,199260.0,-1560.0,71674.0,-0.70,-0.78,35.97,34.92,80.0,-0.02,1.46,3.86,X40,ATH,FMCG
57,NATIONALUM,189.63,247.44,-49.46,H-MC,1.95,101468.0,964.0,29679.0,1.66,0.96,29.25,30.49,79.0,0.03,0.74,33.68,MH,ATH,METALS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,ITC,411.90,452.0,-37.92,H-LC,2.95,148821.0,-1111.0,15701.0,0.63,-0.74,10.55,9.74,4.0,-0.07,1.09,5.45,X40,NTT,FMCG
21,CIPLA,1492.70,1795.0,-20.12,H-LC,3.08,213008.0,8508.0,32910.0,-0.66,4.16,15.45,20.25,10.0,0.26,1.56,13.26,X40N,BTT,PHARMA
58,NESTLEIND,1134.30,1377.0,-12.30,H-LC,3.61,269989.0,4563.0,52216.0,0.85,1.72,19.34,21.40,11.0,0.09,1.98,9.18,XY25,NTT,FMCG
54,LTIM,5564.16,7230.2,-7.23,H-LC,3.71,184176.0,-16134.0,76120.0,0.52,-8.05,41.33,29.94,16.0,-0.21,1.35,27.24,X200,ATH,IT
5,ASIANPAINT,2961.56,4250.0,-10.05,H-LC,4.21,175542.0,-25844.0,113453.0,-0.23,-12.83,64.63,43.51,27.0,-0.23,1.29,22.19,X40,BTT,PAINTS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,LAOPALA,369.40,464.0,122.55,H-SC,2.12,70611.0,-30235.0,56058.0,6.26,-29.98,79.39,25.61,142.0,-0.54,0.52,30.92,AR,NTT,CERAMICS
26,DEN,52.79,75.0,112.44,M-SC,2.74,59568.0,-24896.0,60432.0,5.08,-29.48,101.45,42.07,237.0,-0.41,0.44,26.50,AR,NTT,ENTERTAINMENT
6,ASIANTILES,75.41,137.0,7033.33,L-SC,7.02,78260.0,-15550.0,92167.0,4.80,-16.58,117.77,81.67,269.0,-0.17,0.57,51.55,XR,NTT,CERAMICS
11,BANDHANBNK,214.25,400.0,-6.48,H-SC,8.13,202060.0,-46470.0,261930.0,4.21,-18.70,129.63,86.70,151.0,-0.18,1.48,34.90,XY24,NTT,BANKS
67,SAMMAANCAP,170.35,326.0,-164.77,M-SC,6.64,74082.0,-28128.0,121517.0,4.19,-27.52,164.03,91.37,208.0,-0.23,0.54,20.26,XY25,NTT,FINANCE


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,CAMS,3643.00,5211.76,-5.61,H-SC,4.64,107380.0,5376.0,38549.0,-1.59,5.27,35.90,43.06,122.0,0.14,0.79,23.29,X40N,ATH,MISC
75,STARHEALTH,551.56,761.00,7.30,H-SC,1.59,159647.0,-40569.0,116590.0,-1.53,-20.26,73.03,37.97,144.0,-0.35,1.17,28.18,XY24,NTT,INSURANCE
29,EASEMYTRIP,18.12,26.40,-8.10,M-SC,25.89,88505.0,-88310.0,169107.0,-1.52,-49.94,191.07,45.70,197.0,-0.52,0.65,2.95,XY24,NTT,TRAVEL
76,SURYODAY,189.78,240.00,40.35,H-SC,5.20,100330.0,-49596.0,89274.0,-1.50,-33.08,88.98,26.46,135.0,-0.56,0.74,28.53,XR,NTT,BANKS
89,VALIANTORG,512.64,838.00,-323.33,H-SC,16.01,94926.0,-38360.0,122958.0,-1.38,-28.78,129.53,63.47,139.0,-0.31,0.70,59.12,XR,NTT,CHEMICALS


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,TTKPRESTIG,769.29,770.0,101.49,M-SC,9.86,86853.0,-13924.0,14018.0,2.94,-13.82,16.14,0.09,245.0,-0.99,0.64,14.11,OX40N,NTT,DURABLES
43,INDIGOPNTS,1407.73,1408.0,121.54,M-SC,2.25,145105.0,-29454.0,29485.0,2.64,-16.87,20.32,0.02,221.0,-1.00,1.06,25.83,OX40N,NTT,PAINTS
85,UJJIVANSFB,52.77,60.0,83.20,H-SC,16.87,116046.0,-26433.0,45954.0,-0.30,-18.55,39.60,13.70,163.0,-0.58,0.85,36.97,OX40N,NTT,BANKS
20,CERA,7989.07,9475.0,-18.14,H-SC,3.02,116946.0,-26857.0,53608.0,-0.03,-18.68,45.84,18.60,149.0,-0.50,0.86,29.05,OX40N,NTT,CERAMICS
50,KANSAINER,299.63,340.0,-68.16,H-SC,3.13,219213.0,-50454.0,86786.0,2.83,-18.71,39.59,13.47,138.0,-0.58,1.61,10.87,XY24,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TCS,3794.03,4998.00,-28.69,H-LC,16.19,235342.0,-60592.0,154502.0,0.20,-20.47,65.65,31.73,1.0,-0.39,1.73,0.47,X40,BTT,IT
45,INFY,1461.46,2275.00,-21.39,H-LC,11.27,257008.0,-4593.0,150221.0,-0.82,-1.76,58.45,55.67,3.0,-0.03,1.88,4.24,X40,BTT,IT
47,ITC,411.90,452.00,-37.92,H-LC,2.95,148821.0,-1111.0,15701.0,0.63,-0.74,10.55,9.74,4.0,-0.07,1.09,5.45,X40,NTT,FMCG
90,VBL,492.64,671.64,-10.56,H-LC,9.78,271556.0,6516.0,89776.0,0.99,2.46,33.06,36.33,5.0,0.07,1.99,16.09,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-41.71,H-LC,11.26,197866.0,-18653.0,111557.0,0.71,-8.61,56.38,42.91,7.0,-0.17,1.45,3.62,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,LALPATHLAB,2838.53,3545.0,-22.49,H-SC,11.63,231683.0,38663.0,9383.0,-0.63,20.03,4.05,24.89,137.0,4.12,1.70,48.05,X40N,NTT,HEALTHCARE
56,MEDANTA,1087.93,1486.0,11.18,H-MC,13.93,158700.0,32500.0,13680.0,1.85,25.75,8.62,36.59,108.0,2.38,1.16,41.49,XY24,NTT,HEALTHCARE
28,DMART,3764.92,5201.0,-8.99,H-LC,7.18,227272.0,42791.0,27568.0,1.76,23.20,12.13,38.14,38.0,1.55,1.67,36.26,X40N,NTT,FMCG


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,KPIGREEN,497.21,731.35,13.24,H-SC,11.51,135059.0,9762.0,49243.0,4.05,7.79,36.46,47.09,141.0,0.20,0.99,69.04,MH,ATH,POWER
15,BLUESTARCO,1646.70,2326.38,11.93,H-MC,6.23,193110.0,28440.0,39530.0,0.80,17.27,20.47,41.28,89.0,0.72,1.42,26.63,X40N,ATH,AC
56,MEDANTA,1087.93,1486.00,11.18,H-MC,13.93,158700.0,32500.0,13680.0,1.85,25.75,8.62,36.59,108.0,2.38,1.16,41.49,XY24,NTT,HEALTHCARE
91,VOLTAS,1278.28,1918.49,-1.64,H-MC,7.59,206475.0,14733.0,81289.0,2.02,7.68,39.37,50.08,99.0,0.18,1.51,15.00,XY25,ATH,AC
35,HEROMOTOCO,4311.81,5949.07,-3.00,H-MC,9.48,179718.0,28805.0,28503.0,3.03,19.09,15.86,37.97,93.0,1.01,1.32,49.22,AR,ATH,AUTO


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-3.00,H-MC,9.48,179718.0,28805.0,28503.0,3.03,19.09,15.86,37.97,93.0,1.01,1.32,49.22,AR,ATH,AUTO
51,KPIGREEN,497.21,731.35,13.24,H-SC,11.51,135059.0,9762.0,49243.0,4.05,7.79,36.46,47.09,141.0,0.20,0.99,69.04,MH,ATH,POWER
42,INDIAMART,2327.09,4850.92,-50.91,H-SC,11.87,137122.0,13786.0,119982.0,0.56,11.18,87.50,108.45,119.0,0.11,1.01,36.03,AR,ATH,MISC
92,WHIRLPOOL,1219.98,2270.00,-42.94,M-SC,4.77,96375.0,4876.0,73871.0,-1.15,5.33,76.65,86.07,223.0,0.07,0.71,39.42,XR,NTT,DURABLES
57,NATIONALUM,189.63,247.44,-49.46,H-MC,1.95,101468.0,964.0,29679.0,1.66,0.96,29.25,30.49,79.0,0.03,0.74,33.68,MH,ATH,METALS


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-49.70,L-SC,21.45,74669.0,-38880.0,78933.0,2.15,-34.24,105.71,35.27,268.0,-0.49,0.55,82.75,XR,NTT,HOTELS
6,ASIANTILES,75.41,137.00,7033.33,L-SC,7.02,78260.0,-15550.0,92167.0,4.80,-16.58,117.77,81.67,269.0,-0.17,0.57,51.55,XR,NTT,CERAMICS
3,ALKYLAMINE,2347.98,4546.37,-2.82,H-SC,15.48,90369.0,-10594.0,105126.0,0.76,-10.49,116.33,93.63,148.0,-0.10,0.66,34.60,SR,ATH,CHEMICALS
55,MASFIN,326.60,399.50,-16.89,H-SC,12.75,94200.0,-3780.0,25651.0,2.35,-3.86,27.23,22.32,152.0,-0.15,0.69,36.62,XR,ATH,FINANCE
89,VALIANTORG,512.64,838.00,-323.33,H-SC,16.01,94926.0,-38360.0,122958.0,-1.38,-28.78,129.53,63.47,139.0,-0.31,0.70,59.12,XR,NTT,CHEMICALS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-3.00,H-MC,9.48,179718.0,28805.0,28503.0,3.03,19.09,15.86,37.97,93.0,1.01,1.32,49.22,AR,ATH,AUTO
28,DMART,3764.92,5201.00,-8.99,H-LC,7.18,227272.0,42791.0,27568.0,1.76,23.20,12.13,38.14,38.0,1.55,1.67,36.26,X40N,NTT,FMCG
52,LALPATHLAB,2838.53,3545.00,-22.49,H-SC,11.63,231683.0,38663.0,9383.0,-0.63,20.03,4.05,24.89,137.0,4.12,1.70,48.05,X40N,NTT,HEALTHCARE
5,ASIANPAINT,2961.56,4250.00,-10.05,H-LC,4.21,175542.0,-25844.0,113453.0,-0.23,-12.83,64.63,43.51,27.0,-0.23,1.29,22.19,X40,BTT,PAINTS
4,APOLLOHOSP,7098.95,8285.00,-13.64,H-LC,6.60,172458.0,16281.0,9813.0,0.34,10.42,5.69,16.71,22.0,1.66,1.26,29.68,X40N,BTT,HEALTHCARE


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.44
1,25,40.97
2,50,70.49


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.62
LC    35.39
MC    26.02
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.18
X40N     15.30
X40      14.08
XY25     11.57
XR        8.86
AR        8.32
OX40N     7.35
X5K       2.20
MH        1.73
X200      1.35
SR        1.09
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.24
H-SC    24.03
H-MC    23.13
M-SC    13.12
M-LC     7.10
M-MC     2.56
L-SC     1.47
L-LC     1.05
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.12,-0.56,31.35
IT,11.55,-18.47,77.63
FINANCE,8.02,-19.12,63.19
BANKS,7.46,-12.01,59.57
MISC,5.73,-17.81,85.34
PAINTS,5.58,-14.65,37.24
HEALTHCARE,5.15,6.59,17.00
ELECTRICAL,4.66,-10.56,58.44
AUTO,4.55,-8.98,56.39


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2792959.0,23
XR,1086593.0,13
AR,1076580.0,10
X40,817184.0,11
X40N,519747.0,12
XY25,470082.0,8
OX40N,456029.0,10
SR,194475.0,2
X5K,138714.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2742361.0,28
M-SC,1457963.0,17
H-MC,1221954.0,18
H-LC,1185051.0,19
M-LC,409097.0,5
M-MC,329172.0,2
L-SC,260095.0,3
L-MC,57718.0,1
L-LC,43994.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,923783.0,6
M-SC,XY24,860948.0,7
H-SC,AR,659187.0,5
H-LC,X40,529661.0,6
H-SC,XR,515881.0,6
H-MC,XY24,478246.0,5
M-MC,XY24,329172.0,2
H-LC,AR,232241.0,2
H-MC,X40,207550.0,4


In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
